# PRÁCTICA 10: PERCEPTRÓN MULTICAPA

In [2]:
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score as ac
import time

In [3]:
data = fetch_openml(name='thyroid-allhypo')

In [4]:
data

{'data': array([[ 41.      ,   0.      ,   0.      , ...,   1.14    ,   1.      ,
         109.      ],
        [ 23.      ,   0.      ,   0.      , ...,   0.997912,   0.      ,
         110.787984],
        [ 46.      ,   1.      ,   0.      , ...,   0.91    ,   1.      ,
         120.      ],
        ...,
        [ 75.      ,   1.      ,   0.      , ...,   0.8     ,   1.      ,
         183.      ],
        [ 60.      ,   0.      ,   0.      , ...,   0.83    ,   1.      ,
         121.      ],
        [ 81.      ,   0.      ,   0.      , ...,   0.99    ,   1.      ,
         115.      ]]),
 'target': array(['3', '1', '1', ..., '1', '1', '5'], dtype=object),
 'frame': None,
 'feature_names': ['V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6',
  'V7',
  'V8',
  'V9',
  'V10',
  'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19',
  'V20',
  'V21',
  'V22',
  'V23',
  'V24',
  'V25',
  'V26'],
 'target_names': ['Class'],
 'DESCR': "General Description of Thyroid Dis

In [5]:
X = data.data
# Normalizar
X = MinMaxScaler().fit_transform(X)

y = data.target
X.shape, y.shape

((2800, 26), (2800,))

#### MLP clasificador con validación cruzada (k=10)

In [6]:
# Creación de 10 particiones para realizar validación cruzada
skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
# Array para almacenar los resultados por tamaño de capa oculta
results_summary = np.zeros((10,3), dtype='float')
size = 0
for i in range(10,110,10):
    # Array para almacenar los resultados de las 10 particiones de cross-validation para cada tamaño de capa oculta
    results_per_size = np.zeros((10,3), dtype='float')
    k = 0
    print("\033[1mTamaño de capa:\033[0m",i)
    
    # Creación, entrenamiento y predicción del clasificador MLP para las 10 particiones
    for train_index, test_index in skf.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        start = time.time()
        
        clf = MLPClassifier(hidden_layer_sizes=(i,), max_iter=2000, random_state=0)
        clf.fit(X_train, y_train)
        
        predict_train = clf.predict(X_train)
        predict_test = clf.predict(X_test)
        
        # Tasa de precisión en entrenamiento y en test
        ac_train = ac(y_train, predict_train)
        ac_test = ac(y_test, predict_test)
        
        end = time.time()
        execution_time = end - start
        
        results_per_size[k] = [ac_train,ac_test,execution_time]
        k += 1
    
    # Impresión de resultados para cada tamaño de capa oculta
    print('\33[4mFold     Entrena     Verifica     T.Ejecución\33[0m')
    for i in range(10):
        print(i,'\t %.4f'%results_per_size[i][0],'     %.4f'%results_per_size[i][1],'      %.4f'%results_per_size[i][2])   
    print('\n')
    
    # Resumen de los resultados a través de la media de las precisiones y de la suma total del tiempo empleado
    mean_train = np.mean(results_per_size[:,0], axis = 0, dtype=np.float64)
    mean_test = np.mean(results_per_size[:,1], axis = 0, dtype=np.float64)
    mean_execution_time = np.sum(results_per_size[:,2], axis = 0, dtype=np.float64)
    
    # Impresión del resumen de resultados
    print('MEDIA (Entrena)= %.4f' %mean_train)
    print('MEDIA (Verifica)= %.4f' %mean_test)
    print('Tiempo Total (seg)= %.4f' %mean_execution_time)
    print('\n')
    
    results_summary[size] = [mean_train, mean_test, mean_execution_time]
    size += 1

Tamaño de capa: 10
Fold     Entrena     Verifica     T.Ejecución
0 	 0.7238      0.7500       3.6437
1 	 0.7234      0.7036       2.9706
2 	 0.7234      0.7143       3.1097
3 	 0.7298      0.6750       3.2751
4 	 0.7202      0.6929       2.8962
5 	 0.7262      0.6786       2.6812
6 	 0.7190      0.7107       2.5743
7 	 0.7238      0.6929       2.9728
8 	 0.7198      0.7393       3.0950
9 	 0.7230      0.7357       2.6821


MEDIA (Entrena)= 0.7233
MEDIA (Verifica)= 0.7093
Tiempo Total (seg)= 29.9005


Tamaño de capa: 20
Fold     Entrena     Verifica     T.Ejecución
0 	 0.7218      0.7500       3.1052
1 	 0.7258      0.7071       1.9396
2 	 0.7282      0.7357       2.5146
3 	 0.7313      0.6571       3.2486
4 	 0.7274      0.7071       2.9412
5 	 0.7270      0.6893       2.3641
6 	 0.7242      0.7179       2.9892
7 	 0.7246      0.6964       2.0269
8 	 0.7222      0.7321       2.3585
9 	 0.7250      0.7071       3.5719


MEDIA (Entrena)= 0.7258
MEDIA (Verifica)= 0.7100
Tiempo Total (seg)

#### Resultados

In [7]:
# Impresión de los resultados para cada tamaño de capa oculta
size = 10
print('\33[4mNo.Ocultas     Entrena     Verifica     T.Ejecución\33[0m')
for i in range(10):
    print(size,'\t       %.4f'%results_summary[i][0],'     %.4f'%results_summary[i][1],'      %.4f'%results_summary[i][2])
    size +=10

No.Ocultas     Entrena     Verifica     T.Ejecución
10 	       0.7233      0.7093       29.9005
20 	       0.7258      0.7100       27.0601
30 	       0.7288      0.7096       31.8487
40 	       0.7283      0.7118       28.9778
50 	       0.7325      0.7086       33.1426
60 	       0.7328      0.7100       35.5587
70 	       0.7327      0.7121       30.2342
80 	       0.7312      0.7114       36.8907
90 	       0.7316      0.7100       35.7173
100 	       0.7342      0.7093       33.4752


In [8]:
# Mayor precisión media en el test
best_ac = np.argmax(results_summary[:,1], axis=0)
print("Entendiendo mejor resultado como la mayor precisión media en la prueba del clasificador, obtenemos este con un tamaño de capa oculta de:",(best_ac+1)*10,'(Verifica: %.4f)'%results_summary[best_ac][1])

Entendiendo mejor resultado como la mayor precisión media en la prueba del clasificador, obtenemos este con un tamaño de capa oculta de: 70 (Verifica: 0.7121)


In [9]:
# Menor tiempo de ejecución
best_t = np.argmin(results_summary[:,2], axis=0)
print("El resultado que menor tiempo conlleva de ejecución lo obtenemos con un tamaño de capa de:",(best_t+1)*10, '(T.Ejecución: %.4f)'%results_summary[best_t][2])

El resultado que menor tiempo conlleva de ejecución lo obtenemos con un tamaño de capa de: 20 (T.Ejecución: 27.0601)
